# Introduction

Traditional mean–variance portfolio optimisation is highly sensitive to expected return estimates and often produces unstable allocations in the presence of estimation error.
This project addresses these limitations by integrating systematic factor information within the Black–Litterman (BL) framework, allowing investor views to be incorporated in a disciplined Bayesian manner.

The objectives of this study are:
- to construct a diversified, factor-bearing portfolio,
- to analyse factor behaviour independently prior to optimisation,
- to apply the Black–Litterman model with absolute and relative views,
- to compute optimal allocations under multiple risk preferences,
- and to evaluate portfolio performance and factor attribution via systematic backtesting.

# Portfolio Universe and Data

## Asset Selection

The portfolio universe is chosen to reflect broad economic diversification across multiple asset classes, including equities, fixed income, credit, commodities, and real estate.

## Data Sources

- Asset prices: Yahoo Finance
- Factor returns: Fama–French Data Library
- Frequency: Daily
- Sample length: approximately 2–3 years (>500 observations)
- Returns are computed as excess returns where applicable.

# Factor Definitions and Economic Motivation

This study incorporates multiple systematic factors treated as synthetic investable return streams:
- Market excess return (MKT–RF)
- Size (SMB)
- Value (HML)
- Profitability (RMW)
- Investment (CMA)
- Momentum (MOM)

Each factor represents a distinct and economically motivated source of systematic risk.

## Factor Universe and Economic Motivation
# Factor Study and Systematic Backtesting

## Market Factor (Rm − Rf)
The market excess return (Rm − Rf) is treated as the baseline systematic factor against which all other factor returns are evaluated. It represents the compensation investors receive for bearing aggregate market risk.

Before analysing style and custom factors, we first examine the standalone behaviour of the market factor over the sample period. This includes cumulative performance, volatility, risk-adjusted returns, and drawdowns. Establishing this baseline is essential for interpreting subsequent factor performance and time-varying exposures.

In [ ]:
#| label: mkt-factor-full
#| fig-cap: Market factor performance, cumulative returns, and rolling risk diagnostics (36-month window).
#| fig-align: center
#| echo: false

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from pandas_datareader import data as pdr

sns.set_theme(style="whitegrid")

# ------------------------------------------------------------
# Parameters
# ------------------------------------------------------------
START_DATE = "2007-01-01"
INITIAL_CAPITAL = 1000
ANNUALIZATION = 12          # monthly data
ROLLING_WINDOW = 36         # 3-year window

# ------------------------------------------------------------
# Load Fama–French data
# ------------------------------------------------------------
ff = pdr.DataReader(
    "F-F_Research_Data_5_Factors_2x3",
    "famafrench",
    start=START_DATE
)[0]

ff = ff / 100.0
ff.index = ff.index.to_timestamp()

# ------------------------------------------------------------
# Market excess return (Rm − Rf)
# ------------------------------------------------------------
mkt = ff["Mkt-RF"].rename("MKT")

# ------------------------------------------------------------
# Performance metrics
# ------------------------------------------------------------
mkt_pnl = (1 + mkt).cumprod() * INITIAL_CAPITAL
final_value = mkt_pnl.iloc[-1]

years = len(mkt) / ANNUALIZATION
cagr = (final_value / INITIAL_CAPITAL) ** (1 / years) - 1
vol = mkt.std() * np.sqrt(ANNUALIZATION)
sharpe = mkt.mean() / mkt.std() * np.sqrt(ANNUALIZATION)

drawdown = mkt_pnl / mkt_pnl.cummax() - 1
max_dd = drawdown.min()

performance = pd.DataFrame(
    {
        "Factor": ["Market (Rm − Rf)"],
        "Final Value ($)": [final_value],
        "CAGR (%)": [cagr * 100],
        "Volatility (%)": [vol * 100],
        "Sharpe": [sharpe],
        "Max Drawdown (%)": [max_dd * 100],
    }
)

display(
    performance.style
    .hide(axis="index")
    .format({
        "Final Value ($)": "{:,.0f}",
        "CAGR (%)": "{:.2f}",
        "Volatility (%)": "{:.2f}",
        "Sharpe": "{:.2f}",
        "Max Drawdown (%)": "{:.2f}",
    })
)

# ------------------------------------------------------------
# Plot 1: Cumulative performance
# ------------------------------------------------------------
fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(mkt_pnl.index, mkt_pnl.values, lw=2.2, color="#1f77b4")
ax.text(
    mkt_pnl.index[-1],
    final_value,
    f"  MKT ${final_value:,.0f}",
    fontsize=11,
    va="center"
)

ax.set_ylabel("Portfolio Value ($)")
ax.xaxis.set_major_locator(mdates.YearLocator(2))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.grid(True, axis="y", alpha=0.25)

plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# Rolling volatility & Sharpe
# ------------------------------------------------------------
rolling_std = mkt.rolling(ROLLING_WINDOW).std()
rolling_vol = rolling_std * np.sqrt(ANNUALIZATION)
rolling_sharpe = (
    mkt.rolling(ROLLING_WINDOW).mean()
    / rolling_std.replace(0, np.nan)
    * np.sqrt(ANNUALIZATION)
)

# ------------------------------------------------------------
# Plot 2: Rolling volatility
# ------------------------------------------------------------
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(rolling_vol.index, rolling_vol.values, lw=2, color="#d62728")
ax.set_ylabel("Annualised Volatility")
ax.xaxis.set_major_locator(mdates.YearLocator(2))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.grid(True, axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

# ------------------------------------------------------------
# Plot 3: Rolling Sharpe ratio
# ------------------------------------------------------------
fig, ax = plt.subplots(figsize=(12, 5))

ax.plot(rolling_sharpe.index, rolling_sharpe.values, lw=2, color="#2ca02c")
ax.axhline(0, color="black", lw=1, linestyle="--", alpha=0.6)
ax.set_ylabel("Sharpe Ratio")
ax.xaxis.set_major_locator(mdates.YearLocator(2))
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.grid(True, axis="y", alpha=0.3)

plt.tight_layout()
plt.show()

## Size Factor

In [ ]:
#| label: tbl-smb-performance
#| tbl-cap: SMB factor performance summary ($1,000 base).
#| echo: false
# -----------------------------
# SMB Performance calculations (ISOLATED)
# -----------------------------
smb_pnl = (1 + smb).cumprod() * INITIAL_CAPITAL

smb_final_value = smb_pnl.iloc[-1]
smb_years = len(smb) / ANNUALIZATION
smb_cagr = (smb_final_value / INITIAL_CAPITAL) ** (1 / smb_years) - 1
smb_vol = smb.std() * np.sqrt(ANNUALIZATION)
smb_sharpe = smb.mean() / smb.std() * np.sqrt(ANNUALIZATION)

smb_drawdown = smb_pnl / smb_pnl.cummax() - 1
smb_max_dd = smb_drawdown.min()

smb_performance = pd.DataFrame(
    {
        "Factor": ["Size (SMB)"],
        "Final Value ($)": [smb_final_value],
        "CAGR (%)": [smb_cagr * 100],
        "Volatility (%)": [smb_vol * 100],
        "Sharpe": [smb_sharpe],
        "Max Drawdown (%)": [smb_max_dd * 100],
    }
)

display(
    smb_performance
    .style
    .hide(axis="index")
    .format({
        "Final Value ($)": "{:,.0f}",
        "CAGR (%)": "{:.2f}",
        "Volatility (%)": "{:.2f}",
        "Sharpe": "{:.2f}",
        "Max Drawdown (%)": "{:.2f}",
    })
)